# Лабораторная работа №4

## Хеширование

№ 1. Возьмите реализацию класса HashTable из лекционных материалов и выполните
следующие доработки:
1. Реализуйте квадратичное пробирование как технику повторного хеширования.
2. Реализуйте работу с функцией len (переопределите метод __len__).
3. Реализуйте работу оператора in (переопределите метод __contains__).
4. Переделайте метод put таким образом, чтобы таблица автоматически меняла размер,
когда загрузочный фактор становится больше значения 0.7. Размер должен
увеличиваться примерно в два раза до ближайшего подходящего простого числа.
5. Реализуйте работу оператора del (переопределите метод __delitem__) для удаления
элемента таблицы. Таблица должна автоматически менять размер, когда
загрузочный фактор становится меньше значения 0.2. Размер должен уменьшаться
примерно в два раза до ближайшего подходящего простого числа.
Все выполненные доработки должны быть протестированы.

In [1]:

class HashTable:
    def __init__(self):
        self.size = 11
        self.slots = [None] * self.size
        self.data = [None] * self.size

    def put(self, key, data):
        # Проверка загрузочного фактора
        if self.load_factor() > 0.7:
            self.resize(self.get_next_prime(2 * len(self.slots)))

        hashvalue = self.hashfunction(key, len(self.slots))

        if self.slots[hashvalue] is None:
            self.slots[hashvalue] = key
            self.data[hashvalue] = data
        else:
            if self.slots[hashvalue] == key:
                self.data[hashvalue] = data  # replace
            else:
                nextslot = self.quadratic_rehash(hashvalue, len(self.slots))
                i = 1
                while self.slots[nextslot] is not None and \
                        self.slots[nextslot] != key:
                    nextslot = self.quadratic_rehash(hashvalue, len(self.slots), i)
                    i += 1

                if self.slots[nextslot] is None:
                    self.slots[nextslot] = key
                    self.data[nextslot] = data
                else:
                    self.data[nextslot] = data  # replace

    def load_factor(self):
        return len(self) / len(self.slots)

    def resize(self, new_size):
        old_slots = self.slots
        old_data = self.data

        self.size = new_size
        self.slots = [None] * self.size
        self.data = [None] * self.size

        for i in range(len(old_slots)):
            if old_slots[i] is not None:
                self.put(old_slots[i], old_data[i])

    def get_next_prime(self, n):
        def is_prime(num):
            if num < 2:
                return False
            for i in range(2, int(num**0.5) + 1):
                if num % i == 0:
                    return False
            return True

        while not is_prime(n):
            n += 1
        return n

    def hashfunction(self, key, size):
        """Хеш-функция для строк и других объектов."""
        return hash(key) % size

    def quadratic_rehash(self, oldhash, size, i=1):
        return (oldhash + i**2) % size

    def get(self, key):
        startslot = self.hashfunction(key, len(self.slots))

        data = None
        stop = False
        found = False
        position = startslot
        i = 1
        while self.slots[position] is not None and \
                not found and not stop:
            if self.slots[position] == key:
                found = True
                data = self.data[position]
            else:
                position = self.quadratic_rehash(startslot, len(self.slots), i)
                i += 1
                if position == startslot:
                    stop = True
        return data

    def __getitem__(self, key):
        return self.get(key)

    def __setitem__(self, key, data):
        self.put(key, data)

    def __len__(self):
        return sum(1 for slot in self.slots if slot is not None)

    def __contains__(self, key):
        startslot = self.hashfunction(key, len(self.slots))

        found = False
        stop = False
        position = startslot
        i = 1
        while self.slots[position] is not None and not found and not stop:
            if self.slots[position] == key:
                found = True
            else:
                position = self.quadratic_rehash(startslot, len(self.slots), i)
                i += 1
                if position == startslot:
                    stop = True
        return found

    def __delitem__(self, key):
        startslot = self.hashfunction(key, len(self.slots))

        found = False
        stop = False
        position = startslot
        i = 1
        while self.slots[position] is not None and not found and not stop:
            if self.slots[position] == key:
                found = True
                self.slots[position] = None
                self.data[position] = None
            else:
                position = self.quadratic_rehash(startslot, len(self.slots), i)
                i += 1
                if position == startslot:
                    stop = True

        if not found:
            raise KeyError(f"Key {key} not found in HashTable")

        # Проверка загрузочного фактора
        if self.load_factor() < 0.2:
            new_size = max(11, self.get_next_prime(len(self.slots) // 2))
            self.resize(new_size)

# Пример использования:
H = HashTable()

# Добавление строковых ключей
H["apple"] = "fruit"
H["carrot"] = "vegetable"
H["banana"] = "fruit"
H["dog"] = "animal"

print("After adding elements:")
print("Slots:", H.slots)
print("Data:", H.data)

# Проверка наличия ключей
print("\nIs 'apple' in H?", "apple" in H)  # True
print("Is 'cat' in H?", "cat" in H)  # False

# Получение значения по строковому ключу
print("\nValue for 'apple':", H["apple"])  # "fruit"

# Удаление элемента
del H["apple"]
print("\nAfter deleting 'apple':")
print("Slots:", H.slots)
print("Data:", H.data)

After adding elements:
Slots: ['carrot', None, None, None, None, None, 'apple', 'dog', 'banana', None, None]
Data: ['vegetable', None, None, None, None, None, 'fruit', 'animal', 'fruit', None, None]

Is 'apple' in H? True
Is 'cat' in H? False

Value for 'apple': fruit

After deleting 'apple':
Slots: ['carrot', None, None, None, None, None, None, 'dog', 'banana', None, None]
Data: ['vegetable', None, None, None, None, None, None, 'animal', 'fruit', None, None]


№ 4. Дана строчка русского текста, состоящая из слов и пробелов. Словом считается
последовательность русских букв, слова разделены одним или большим числом пробелов.

Для каждого слова этого текста узнайте порядковый номер его вхождения в текст именно в
той форме, в которой указано слово. Для первого вхождения слова выведите «1», для
второго вхождения того же слова выведите «2» и так далее.

Для решения этой задачи используйте класс HashTable из задания № 3.

In [ ]:
import re

def word_occurrences(text):
    word_table = HashTable()
    result = []

    words = re.findall(r'[а-яА-ЯёЁ]+', text)

    for word in words:
        if word in word_table:
            word_table[word] += 1
        else:
            word_table[word] = 1
        result.append(f"{word}({word_table[word]})")

    return result

text = "Привет мир мир Привет мир"
occurrences = word_occurrences(text)
print(" ".join(occurrences))

Привет(1) мир(1) мир(2) Привет(2) мир(3)


№ 5. Напишите программу, имитирующую процесс регистрации и авторизации. Для каждого
пользователя программа должна сохранять логин, хеш его пароля и «соль». Для хранения
данных можно использовать БД или файл.

Действия при сохранении пароля:

1. Сгенерируйте длинную случайную «соль» при помощи модуля secrets (secrets —
Generate secure random numbers for managing secrets — Python 3.10.0 documentation).
Длина «соли» должна быть такой же как и выходные данные используемой вами
хэш-функции. Например, если для хеширования вы используете SHA256, то на
выходе вы получите 256 бит (32 байта). В этом случае соль должна составлять не
менее 32 случайных байт.
2. Добавьте «соль» к паролю и хэшируйте его с помощью функции scrypt из модуля
hashlib (Хеширование паролей модулем hashlib в Python. (docs-python.ru)).
3. Сохраните логин, «соль» и получившейся хэш в БД или в файл.

Действия при проверке пароля:

1. Извлеките «соль» и хэш пользователя из БД или файла.
2. Добавьте «соль» к введенному паролю и хэшируйте его, используя ту же хэш-
функцию, что и в алгоритме сохранения пароля.
3. Сравните получившейся хэш введенного пароля с хэшом из БД или файла. Если они
совпадают, то пароль правильный. В противном случае пароль был введен неверно.

In [15]:
import secrets
import hashlib
import json

class UserAuthSystem:
    def __init__(self, db_file="users.json"):
        self.db_file = db_file
        try:
            with open(self.db_file, "r") as file:
                self.users = json.load(file)
        except FileNotFoundError:
            self.users = {}

    def save_to_db(self):
        with open(self.db_file, "w") as file:
            json.dump(self.users, file)

    def register(self, username, password):
        if username in self.users:
            print("Пользователь с таким логином уже существует.")
            return False

        salt = secrets.token_bytes(32)  # 32 байта для SHA256
        hashed_password = hashlib.scrypt(password.encode(), salt=salt, n=16384, r=8, p=1)

        self.users[username] = {
            "salt": salt.hex(),
            "password_hash": hashed_password.hex()
        }
        self.save_to_db()
        print("Пользователь успешно зарегистрирован.")
        return True

    def authenticate(self, username, password):
        if username not in self.users:
            print("Пользователь не найден.")
            return False

        salt = bytes.fromhex(self.users[username]["salt"])
        stored_hash = bytes.fromhex(self.users[username]["password_hash"])

        hashed_password = hashlib.scrypt(password.encode(), salt=salt, n=16384, r=8, p=1)

        if hashed_password == stored_hash:
            print("Авторизация успешна.")
            return True
        else:
            print("Неверный пароль.")
            return False

auth_system = UserAuthSystem()

auth_system.register("user1", "password123")
auth_system.register("user2", "securepassword")

auth_system.authenticate("user1", "password123")  # Успешно
auth_system.authenticate("user2", "wrongpassword")  # Неверный пароль
auth_system.authenticate("user3", "password123")  # Пользователь не найден

Пользователь с таким логином уже существует.
Пользователь с таким логином уже существует.
Авторизация успешна.
Неверный пароль.
Пользователь не найден.


False

№ 6. Напишите программу, которая принимает от пользователя путь до директории. Для всех
файлов из данной директории должен быть вычислен хеш. Программа должна выявить и
вывести на экран все дубликаты в этой директории (т.е. файлы, у которых одинаковый хеш).

In [ ]:
import os
import hashlib

def calculate_file_hash(filepath):
    hash_func = hashlib.sha256()
    try:
        with open(filepath, "rb") as file:
            while chunk := file.read(8192):
                hash_func.update(chunk)
    except (OSError, IOError):
        print(f"Не удалось прочитать файл: {filepath}")
        return None
    return hash_func.hexdigest()

def find_duplicates(directory):
    if not os.path.isdir(directory):
        print("Указанный путь не является директорией.")
        return

    hashes = {}
    duplicates = []

    for root, _, files in os.walk(directory):
        for file in files:
            filepath = os.path.join(root, file)
            file_hash = calculate_file_hash(filepath)
            if file_hash:
                if file_hash in hashes:
                    duplicates.append((filepath, hashes[file_hash]))
                else:
                    hashes[file_hash] = filepath

    if duplicates:
        print("Найдены дубликаты:")
        for duplicate, original in duplicates:
            print(f"Дубликат: {duplicate} -> Оригинал: {original}")
    else:
        print("Дубликаты не найдены.")

directory = input("Введите путь до директории: ")
find_duplicates(directory)

Найдены дубликаты:
Дубликат: /Users/pog0sian/Desktop/Технологии параллельных вычислений/Лабораторная работа №7 — копия.docx -> Оригинал: /Users/pog0sian/Desktop/Технологии параллельных вычислений/Лабораторная работа №7.docx
